In [2]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from helpers import *

In [0]:
#Create embeddings with glove_solution
from glove_solution import main as glove_main
glove_main()

In [55]:
from pattern.en import *

In [3]:
word_embeddings = np.load("embeddings.npy")
word_embeddings.shape

(21161, 20)

In [4]:
words_list = words_list("vocab_cut.txt")

In [5]:
tweets_pos_txt = tweets_txt("Datasets/twitter-datasets/train_pos.txt")
tweets_neg_txt = tweets_txt("Datasets/twitter-datasets/train_neg.txt")

# Do some data cleaning

In [29]:
#Remove duplicates
unique_tweets_pos_txt = remove_duplicated_tweets_txt(tweets_pos_txt)
unique_tweets_neg_txt = remove_duplicated_tweets_txt(tweets_neg_txt)

In [61]:
common = ['"', ',', '.', ')', '(', '-', "<user>", "<url>", "a", "the", "of", "to", \
                "it", "this", "that", "these", "there"]
t = [t for t in unique_tweets_pos_txt[70].split(" ") if t not in common]

if(t[0:3] == ['!', '!', '!']):
    t = t[3:]
print(t)
print([conjugate(w) for w in t])
conjugate("dont")

["don't", 'worry', 'spider', 'is', 'smaller', 'than', 'you', 'yeah', '..', 'so', 'is', 'grenade', 'hahaha']
['do', 'worry', 'spider', 'be', 'smaller', 'than', 'you', 'yeah', '..', 'so', 'be', 'grenade', 'hahaha']


'dont'

In [18]:
#Filter tweets
common = ['"', ',', '.', ')', '(', '-', \
                "<user>", "<url>", "a", "the", "of", "to", \
                "it", "this", "that", "these", "there"]
tweets_pos_vec = tweet_means(tweets_pos_txt, word_embeddings, \
                             words_list, word_embeddings[0].shape[0], \
                             true, common)
tweets_neg_vec = tweet_means(tweets_new_txt, word_embeddings, \
                             words_list, word_embeddings[0].shape[0], \
                             true, common)

(100000,)


In [0]:
#Means for tweets not filtered
tweets_pos_vec = tweet_means(tweets_pos_txt, \
    word_embeddings, words_list, word_embeddings[0].shape[0])
tweets_neg_vec = tweet_means(tweets_new_txt, \
    word_embeddings, words_list, word_embeddings[0].shape[0])

In [0]:
np.save('tweets_pos_vec', tweets_pos_vec)
np.save('tweets_neg_vec', tweets_neg_vec)

# TRAINING TIME

In [ ]:
tweets_pos_vec = np.load('tweets_pos_vec.npy')
tweets_neg_vec = np.load('tweets_neg_vec.npy')

In [ ]:
#TO USE IF ALGO ASKS LABEL TO BE 1 AND -1
X = np.concatenate((tweets_pos_vec, tweets_neg_vec))
y = np.concatenate((np.ones((tweets_pos_vec.shape[0])), -1*np.ones((tweets_pos_vec.shape[0]))))

In [0]:
#TO USE IF ALGO ASKS LABEL TO BE 1 AND 0
X = np.concatenate((tweets_pos_vec, tweets_neg_vec))
y = np.concatenate((np.ones((tweets_pos_vec.shape[0])), np.zeros((tweets_pos_vec.shape[0]))))

In [ ]:
#NEED TO CHECK IF IT IS REALLY WHAT SCORING ASKS
def accuracy(estimator, X, y):
    return np.mean(y==estimator.predict(X))

In [ ]:
j = 0.9

train_X = X[0:int(j*len(y))]
train_y = y[0:int(j*len(y))]
test_X = X[int(j*len(y)):]
test_y = y[int(j*len(y)):]

def accuracy(y1, y2):
    return np.mean(y1==y2)

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
mlp.fit(train_X, train_y)
y_pred = mlp.predict(test_X)
accuracy(test_y, y_pred)

In [20]:
numpy.random.seed(42)
clf = ???
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
scores = cross_val_score(estimator=clf, X, y, scoring=accuracy, cv=cv)

SyntaxError: positional argument follows keyword argument (<ipython-input-20-bcdad1a63b29>, line 4)